In [1]:
import pandas as pd
import hashlib


In [2]:
def log_df(df, limit=5):
    print(df.shape)
    return df.head(limit)

# integer columns that have null
int_dtype = {'playlist_position': 'Int64', 'price': 'Int64', 'serial_number':'Int64'}


## fix title on 'Oli.csv so it uses the full video title

In [ ]:
my_oli_df = pd.read_csv('../raw_data/Oli.csv', usecols=['id', 'my id', 'name'])
log_df(my_oli_df)

In [ ]:
oli_yt_df = pd.read_csv('../raw_data/oli_playlist.csv', usecols=['position', 'video_title', 'video_id'])
log_df(oli_yt_df)

In [ ]:
oli_yt_df['temp_title'] = oli_yt_df['video_title'].replace({"^'Oli ": ""}, regex=True)

oli_yt_df.head()

In [ ]:
merge_df = my_oli_df.merge(oli_yt_df, how = 'outer', left_on='name', right_on='temp_title')
merge_df.head()

In [ ]:
merge_df[merge_df['video_id'].isna()].shape

In [ ]:
merge_df.to_csv('../processed_data/draft/oli_fix_title.csv', index=False)

## add unique id for each model

Some Oli ukes use the same model number for different woods, e.g curly Redwood and redwood are labeled as r. I want each model + wood  to have  id

In [ ]:

my_oli_df = pd.read_csv('../raw_data/Oli.csv', dtype=int_dtype)
log_df(my_oli_df)

In [ ]:
my_oli_df['oli_id'] = ''

for i, row in my_oli_df.iterrows():
    if(pd.isna(row['model'])):
        continue

    temp = row['model'] + row['top_wood'] + row['back_wood']
    # if(not pd.isna(row['model_notes'])):
    #     temp = temp + row['model_notes']
    my_oli_df.loc[i, 'oli_id'] = hashlib.md5(temp.encode()).hexdigest()

In [ ]:
log_df(my_oli_df)

In [ ]:
len(my_oli_df['oli_id'].unique())
# 163

In [ ]:
my_oli_df.to_csv('../raw_data/Oli.csv', index=False)

## add published_at to Oli csv

In [ ]:
my_oli_df = pd.read_csv('../raw_data/Oli.csv',dtype=int_dtype)
log_df(my_oli_df)

In [ ]:
cols = ['video_id', 'published_at']

videos_df = pd.read_csv('../processed_data/videos_list.csv', usecols=cols)
videos_df.columns = ['video_published_at', 'video_id']

log_df(videos_df)

In [ ]:
new_oli_df = my_oli_df.merge(videos_df, how='left')
log_df(new_oli_df)

In [ ]:
new_oli_df.to_csv('../raw_data/Oli.csv', index=False)

## update video data 'Oli.csv 

In [3]:
my_oli_df = pd.read_csv('../raw_data/Oli.csv')
log_df(my_oli_df)

(448, 25)


,playlist_position,video_title,model,tier,type,size,top_wood,back_wood,finish,model_notes,...,listing_url,video_id,video_published_at,video_type,video_provider,playlist_id,oli_id,video_description,redirect_url,date
0,1.0,'Oli Mango Tenor Prototype,L2-TMG,L2,TMG,Tenor,Mango,Mango,gloss,NaN,...,NaN,1__pjKICYcA,2023-08-21T22:57:21Z,other,youtube,PLEQbDXy0ShuW7JotVhROvR4TxhmO_0tLD,f8e2f0955694a60036f37491c15bcca4,NaN,NaN,NaN
1,2.0,"""Hawai'i '78"" 'Oli Mango Tenor & Pono Master S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,mgGgRXIZY4Q,2023-08-21T23:00:36Z,song,youtube,PLEQbDXy0ShuW7JotVhROvR4TxhmO_0tLD,NaN,NaN,NaN,NaN
2,3.0,'Oli Mango Concert L2 MGC,L2-CMG,L2,CMG,Concert,Mango,Mango,gloss,NaN,...,NaN,Oj_48kO9R8w,2023-11-16T18:57:56Z,sound_sample,youtube,PLEQbDXy0ShuW7JotVhROvR4TxhmO_0tLD,b896dea775786969dd6a0305484d3e8b,NaN,NaN,NaN
3,4.0,'Oli Mango Baritone L2 MGB,L2-BMG,L2,BMG,Baritone,Mango,Mango,gloss,NaN,...,NaN,K1WS2LC0U2s,2023-11-16T19:12:58Z,sound_sample,youtube,PLEQbDXy0ShuW7JotVhROvR4TxhmO_0tLD,6c3bbc40618711d4c968ccc0b716d915,NaN,NaN,NaN
4,5.0,Danny's Song on 'Oli Mango Baritone & Tenor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,sMBVxO3s9KU,2023-11-16T19:28:57Z,song,youtube,PLEQbDXy0ShuW7JotVhROvR4TxhmO_0tLD,NaN,NaN,NaN,NaN


In [4]:
cols = ['position','video_title','video_id','playlist_id', 'published_at']

videos_df = pd.read_csv('../processed_data/videos_list.csv', usecols=cols)

log_df(videos_df)

(390, 5)


,position,video_title,published_at,video_id,playlist_id
0,1,'Oli L1 MGBG Mango Baritone,2024-01-30T08:53:06Z,uBJy6pgpNIU,PLEQbDXy0ShuWQ4iKD4YVC1iOj3uKY1R-2
1,2,O.C. Bear Custom Lahaina Guitar & ‘Oli Cedar E...,2024-02-01T06:44:56Z,aFgun3_F1fQ,PLEQbDXy0ShuWQ4iKD4YVC1iOj3uKY1R-2
2,3,'Oli L2 TCE Cedar Macassar Ebony Tenor,2024-02-01T06:46:48Z,mSw_GV9LxOM,PLEQbDXy0ShuWQ4iKD4YVC1iOj3uKY1R-2
3,4,'Oli Cedar Macassar L2 Baritone,2024-02-28T22:39:41Z,uDCKmlLujrs,PLEQbDXy0ShuWQ4iKD4YVC1iOj3uKY1R-2
4,5,'Oli Spruce Rosewood L2 Baritone,2024-02-28T22:44:55Z,TkO2YRW2vQE,PLEQbDXy0ShuWQ4iKD4YVC1iOj3uKY1R-2


In [5]:
new_oli_df = my_oli_df[pd.isna(my_oli_df['video_title']) & pd.notna(my_oli_df['video_id']) ]
log_df(new_oli_df)

(0, 25)


,playlist_position,video_title,model,tier,type,size,top_wood,back_wood,finish,model_notes,...,listing_url,video_id,video_published_at,video_type,video_provider,playlist_id,oli_id,video_description,redirect_url,date


In [6]:
ids = new_oli_df['video_id'].unique()
ids

array([], dtype=object)

In [7]:
for i, row in videos_df.iterrows():
    video_id = row['video_id']
    if(video_id not in ids):
        continue

    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'playlist_position'] = row['position']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_title'] = row['video_title']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'playlist_id'] = row['playlist_id']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_type'] = 'sound_sample'
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_provider'] = 'youtube'


In [8]:
# new_oli_df = new_oli_df.astype({"playlist_position": int})

log_df(new_oli_df)

(0, 25)


,playlist_position,video_title,model,tier,type,size,top_wood,back_wood,finish,model_notes,...,listing_url,video_id,video_published_at,video_type,video_provider,playlist_id,oli_id,video_description,redirect_url,date
